In [1]:
# import block
from gensim.models import word2vec
import pandas as pd
import json
from pathlib import Path
import string
import re
import pickle
from itertools import chain
from fuzzywuzzy import fuzz, process
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


In [2]:
#first load all recipe data
BOrecipes = []
BOingredients = []
BOids = []
BOpath = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(BOpath.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        BOrecipes.append(r)
        BOing = r.get('Ingredients')
        BOid = r.get('RecipeID')
        BOids.append(id)
        if BOing is not None:
            BOingredients.append([lemmatizer.lemmatize(re.sub(r'[^a-z ]', '', i['Name'].lower().strip()))
                                  for i in BOing if i is not None and i['Name'] is not None])

                
                

/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-68-87332.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-71-87778.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-35-191-157959.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-76-83346.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-74-82-565669.json
/Users/bridgitboulahanis/Dropbox

In [3]:
BOingredients

[['garam masala',
  'fresh coriander leaves',
  'fresh ginger root',
  'turmeric',
  'water',
  'ground coriander',
  'water',
  'ground cumin',
  'red pepper hot',
  'rice',
  'tea leaves',
  'chana dal',
  'salt',
  'ghee',
  'cumin seed',
  'onion'],
 ['pasta',
  'shrimp',
  'butter',
  'fresh italian parsley',
  'red pepper flakes',
  'ground black pepper',
  'kosher salt',
  'dry white wine',
  'garlic',
  'parmesan cheese grated',
  'lemon'],
 ['black pepper',
  'egg',
  'for hush puppies',
  'whole grain',
  'onion',
  'salt',
  'salt',
  'for the fish',
  'buttermilk',
  'baking soda',
  'baking powder',
  'corn flour',
  'cayenne',
  'fish'],
 ['chicken tenderloins or  chicken breast filets',
  'sweet onion',
  'honeu barbeque sauce',
  'sugar',
  'powdered garlic',
  'black ground pepper',
  'sea salt'],
 ['chicken breasts boneless and skinless',
  'garlic',
  'shallots minced',
  'thai chilies',
  'ground white pepper',
  'egg',
  'vegetable oil',
  'soy sauce',
  'thai basi

In [4]:
#now we want to load spoonacular data and create a dataframe that has ingredients and their aisle 
# this will be our classification data

SArecipes = []
SAingredients = []
SAingclass = []
SAnoise = []
SAingID = []
SAmeta = []
SAmeasure = []

seenID = set()
SApath = Path('~/Dropbox/Not_work/insight/greenmarkets/spoonac/data/')
for i, po in enumerate(SApath.expanduser().glob('*/*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        for rec in r:
            currenting = rec['extendedIngredients']
            for ing in currenting:
                SAnoise.extend(i.strip() for i in ing['metaInformation'])
                if ing['id'] in seenID:
                    continue
                seenID.add(ing['id']) 
                SAingredients.append(ing['name'])
                SAingclass.append(ing['aisle'])  
                SAingID.append(ing['id'])
                SAmeasure.append(ing['measures']['us']['unitLong'])
                SAmeta.extend([ing['meta']])

                    





In [5]:
#grab all the ingrdients from spoonacular and put in dataframe
classifying = pd.DataFrame({'ingredient': SAingredients, 'aisle': SAingclass, 'meta': SAmeta, 'measure': SAmeasure,  'id': SAingID})


,ingredient,aisle,meta,measure,id
0,bell pepper,Produce,"[seeded, chopped]",large,10211821.0
1,canned coconut milk,Canned and Jarred,[canned],ounces,12117.0
2,carrots,Produce,"[peeled, chopped]",larges,11124.0
3,cornstarch,Baking,[],Tbsps,20027.0
4,curry powder,Spices and Seasonings,[],Tbsp,2015.0
...,...,...,...,...,...
1949,aji amarillo,Produce;Ethnic Foods,[cut into 1/4-inch dice*],Tbsps,10311819.0
1950,aji amarillo paste,Ethnic Foods,[],teaspoon,1006973.0
1951,yuca,Produce,"[fresh, cut into big pieces]",pound,11134.0
1952,cedar plank,Grilling Supplies;Gourmet,[],,11911111.0


In [33]:
#some words you noticed should get added to SAnoise
wta = ['ground', 'taste', 'ounce', 'large']
SAnoise.extend(wta)
SAnoise = set(SAnoise)
#remember you can check if it is in there with: 'fresh' in SAnoise

AttributeError: 'set' object has no attribute 'extend'

In [13]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 
#here we are removing meta words and stop words
stop_words = set(stopwords.words('english')) 

def removenoise(input_ing):
    assert type(input_ing) == list, type(input_ing)
    noise_free_ing = []
    for word in input_ing:
        checked = []
        splitit = word.split()
        checked.extend(i for i in splitit if i not in SAnoise and i not in stop_words)
        noise_free_ing.append(' '.join(checked))
    return noise_free_ing
        


In [14]:
#call the dedupe functions
nning = []
for i in BOingredients:
    nning.append(removenoise(i))



In [27]:
# classify ingredients into aisles
whichaisle = {}
seen = set()
notin = []
for i in nning:
    for j in i:
        if j in seen:
            continue
        seen.add(j) 
#        a = classifying.index[classifying['ingredient'] == j].tolist()        
        highest = process.extractOne(j,classifying['ingredient'])
        a = classifying.index[classifying['ingredient'] == highest[0]]
        if highest[1] > 80:
            whichaisle[j] = classifying['aisle'][a].tolist()
        else:
            whichaisle[j]  = None
            print(f'No match for {j}, closest was {highest[0]} at {highest[1]}')



No match for hush puppies, closest was peas at 68


No match for , closest was bell pepper at 0
No match for tumeric, closest was rice at 77
No match for mooli, closest was semolina flour at 72
No match for evoo, closest was provolone at 68
No match for leak, closest was leek at 75
No match for taragon, closest was fresh tarragon at 77
No match for unsulfured molasses, closest was red miso at 68
No match for removed, closest was creamed coconut at 66
No match for pounds, closest was round steak at 75
No match for option, closest was onion at 73
No match for thyme garnish, closest was thyme sprigs at 72
No match for plume, closest was plums at 80
No match for furikaki seaweed, closest was dill seed at 60
No match for pinch asafetida optional, closest was spinach at 61
No match for sake, closest was sesame oil at 68
No match for courgette, closest was corn at 68
No match for cooking spray, closest was coke at 68
No match for rustic rub, closest was Spice Rub at 74
No match for tater tots, closest was potato tots at 76
No match for tacos, 

No match for bunch sage, closest was ground sage at 67
No match for oz package rigatoni, closest was lager at 72
No match for oz sw premium ready recipe, closest was bread at 72
No match for bunsrolls, closest was sole at 68
No match for sake cooking sake ok, closest was coke at 68
No match for garnish optional, closest was fish at 68
No match for country crock, closest was country ham at 74
No match for curly patsa, closest was pasta at 72
No match for basil optional, closest was thai basil at 64
No match for goose, closest was cooked jasmine rice at 54
No match for tbs thyme leaves, closest was lettuce leaves at 67
No match for classic coca cola, closest was clams at 72
No match for saffron tumeric, closest was ground turmeric at 73
No match for marinade ingredients, closest was mirin at 72
No match for gremolata, closest was granola at 62
No match for g strong chesse, closest was string cheese at 79
No match for sait, closest was salt at 75
No match for reducedfat feta, closest was 

No match for oleo, closest was guacamole at 77
No match for jaggery, closest was lager at 67
No match for hijiki, closest was dried shiitake mushrooms at 60
No match for papper flakes, closest was red pepper flakes at 80
No match for special equipment, closest was speck at 72
No match for saute together, closest was skate at 72
No match for also take, closest was salmon steaks at 64
No match for mrs dash, closest was dashi at 80
No match for bechamel, closest was caramel at 67
No match for retsina, closest was thousand island dressing at 64
No match for cummin, closest was ground cumin at 75
No match for lasagne sheets, closest was fresh lasagna sheets at 76
No match for pillsbury crescent dinner, closest was dill at 68
No match for omelet, closest was millet at 67
No match for wonton wrappers, closest was capers at 75
No match for fabes, closest was mixed vegetables at 72
No match for compango, closest was mango at 77
No match for piment espelette, closest was pimento at 79
No match f

No match for bltterteig aus dem khlregal rolle, closest was butter at 75
No match for kleine eier, closest was kale at 68
No match for kse gratinkse, closest was raisins at 64
No match for krutersalz, closest was salt at 68
No match for pfeffer, closest was paneer at 62
No match for ei zum bestreichen, closest was rice at 68
No match for ingredients, closest was greens at 71
No match for wet ingredients, closest was beet greens at 69
No match for carrot strips, closest was carrots at 77
No match for strips dont like ne, closest was strip loin at 72
No match for colavita gnocchi, closest was potato gnocchi at 73
No match for pepperonchinis, closest was pepperoni at 80
No match for ritz, closest was grapefruit at 68
No match for garden veggie ritz, closest was eggs at 68
No match for stephen ceideburg, closest was tempeh at 60
No match for heavyduty aluminum foil, closest was haloumi at 64
No match for new mexico chiles, closest was chives at 75
No match for cooking sprayoil, closest was

...

No match for pig ears, closest was pears at 77
No match for pig nose, closest was pickled jalapenos at 64
No match for mesquite smoke, closest was squid at 72
No match for mild pimentn, closest was pimento at 77
No match for jack sharp c heddar, closest was sharp cheddar cheese at 73
No match for gekookte bieten, closest was coke at 68
No match for gerookte markreel, closest was mackerel at 68
No match for rode wijnazijn, closest was wine at 68
No match for bieslook, closest was beef stock at 56
No match for parsely stems, closest was parsley at 77
No match for panini loaf, closest was tahini at 60
No match for thinly optional, closest was salt at 64
No match for umeboshi, closest was rum at 60
No match for leg quarters, closest was lambs quarters at 77
No match for dijonstyle, closest was dijon mustard at 61
No match for pounds ballpark franks, closest was pork at 68
No match for pounds pig tails, closest was salsa at 60
No match for nature, closest was turkey at 67
No match for lifti

No match for persilja, closest was jam at 72
No match for grdde, closest was shredded cheddar cheese at 72
No match for freshly asiago, closest was fresh sage at 75
No match for confit, closest was corn at 68
No match for suckling pig, closest was duck at 68
No match for frogmat, closest was meat at 68
No match for ichimi togarashi optional, closest was shichimi togarashi at 74
No match for veloute, closest was vermouth at 67
No match for deshelled prawns, closest was shell pasta at 67
No match for calamari rings, closest was mirin at 72
No match for tamatoe concentrate, closest was limeade concentrate at 79
No match for inchthick, closest was bone in chicken breast at 70
No match for california, closest was corn at 68
No match for cod cheeks, closest was cheese at 75
No match for samphire, closest was jam at 60
No match for allsipce, closest was ground allspice at 79
No match for rmoulade go bonappetitcom recipe may, closest was apple at 72
No match for rindspltzli, closest was spelt 

No match for preferably wholewheat, closest was sole at 68
No match for lala, closest was garam masala at 68
No match for using diamond crystal, closest was salt at 68
No match for sprigs sage, closest was figs at 68
No match for cm square, closest was graham cracker squares at 76
No match for ml bottle, closest was bottled garlic at 67
No match for sucanant, closest was yuca at 68
No match for zatar see note, closest was agar agar at 64
No match for osso buco, closest was boston butt at 60
No match for tool, closest was beetroot at 68
No match for wooden spoon, closest was golden syrup at 58
No match for small bowl, closest was salt at 68
No match for spatula, closest was salt at 68
No match for parmiagiannoreggiano, closest was eggs at 68
No match for pound loaf, closest was ground lamb at 67
No match for mlieko, closest was lime at 68
No match for vajce cel, closest was caramel sauce at 61
No match for garden combination ragu, closest was agar agar at 64
No match for stalk roughly, 

No match for scrambled, closest was lamb at 68
No match for wet mop optional, closest was ale at 72
No match for birdseye chillies, closest was red chilies at 74
No match for melange trois, closest was lager at 72
No match for packet, closest was water-packed tuna at 75
No match for outer wrapper, closest was apple at 72
No match for filling garnish, closest was fish at 68
No match for maida glass, closest was glass noodles at 59
No match for chipotle marinade, closest was mirin at 72
No match for approximate nutrition facts, closest was nuts at 68
No match for spun torn, closest was corn at 68
No match for anything else would like, closest was lime at 68
No match for subrolls, closest was dinner yeast rolls at 68
No match for chipotles adobe, closest was chipotle paste at 76
No match for see instr, closest was black mustard seeds at 67
No match for broiler, closest was sole at 68
No match for matchstick, closest was meat at 68
No match for italianstyle tomaotes, closest was tomato at 

No match for rotelle corkscrews, closest was corn at 68
No match for oz italianstyle, closest was pita at 68
No match for seafood cocktail, closest was seafood stock at 76
No match for provola affumicata, closest was provolone at 64
No match for ortega chilis, closest was red chilies at 75
No match for cook time hour, closest was lime at 68
No match for serves least people, closest was peas at 68
No match for sumak, closest was ground sumac at 72
No match for poussin, closest was langoustines at 64
No match for individual larger bags doritos, closest was lager at 72
No match for warm baguette, closest was baguettes at 73
No match for equipment panini press, closest was lime at 68
No match for g leicester, closest was rice at 68
No match for molho, closest was guacamole at 60
No match for veggies would like add, closest was lime at 68
No match for eggplant optional, closest was eggs at 68
No match for boeuf de ronde, closest was beef tenderloin at 62
No match for oignons moyens, closest

In [29]:
#Actual W2V modeling
# Set values for NN parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 3    # 50% of the corpus                    
num_workers = 4       # Number of CPUs
context = 10          # Context window size; 
downsampling = 1e-3   # threshold for configuring which 
                    # higher-frequency words are randomly downsampled

# Initialize and train the model 
model = word2vec.Word2Vec(nning, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [34]:
#output both the w2v model and the aisle classification dict
pickle.dump(model,open("bigoven/w2vmodel.pkl","wb"))
pickle.dump(whichaisle,open("bigoven/aisleclassification.pkl","wb"))
pickle.dump(SAnoise, open('bigoven/noiselist.pkl', 'wb'))

In [70]:
#proof of concept, also not a production cell
print(model.wv.most_similar('feta', topn=50))
print('=============')
print(model.wv.most_similar('oil cured olives'))

[('oil cured olives', 0.9911516904830933), ('spinach leaves', 0.9907946586608887), ('sun tomatoes oil', 0.990415096282959), ('fusilli', 0.9873874187469482), ('turns around pan extravirgin olive oil', 0.9861507415771484), ('grilled chicken breasts', 0.9853078722953796), ('roasted bell pepper', 0.9838919639587402), ('flatbread', 0.9830859899520874), ('ciabatta bread', 0.9828497171401978), ('grinds pepper', 0.9823001623153687), ('plum tomato', 0.9815158843994141), ('fusilli pasta', 0.9811870455741882), ('focaccia bread', 0.9804506301879883), ('curly kale', 0.9802354574203491), ('crusty bread', 0.9801878333091736), ('tomatoes thinly', 0.979414701461792), ('zucchini summer squash', 0.9790729284286499), ('cheese ravioli', 0.9789094924926758), ('dough bread', 0.9783138036727905), ('reducedfat cream cheese', 0.9776568412780762), ('basil pesto', 0.9768462181091309), ('greek seasoning', 0.9760223627090454), ('artichoke hearts water', 0.9743931293487549), ('artichokes', 0.9740794897079468), ('her

In [36]:
print(fuzz.ratio('parmesan cheese', 'cheddar cheese'))
print(fuzz.partial_ratio('parmesan cheese', 'cheddar cheese'))
print(fuzz.token_sort_ratio('parmesan cheese', 'cheddar cheese'))


62
64
48


In [25]:
str2Match = "limes"
strOptions = classifying['ingredient']
#Ratios = process.extract(str2Match,strOptions)
#print(Ratios)
# You can also select the string with the highest matching percentage
highest = process.extractOne(str2Match,strOptions)
print(highest)


('lime', 89, 176)


In [22]:
highest[0]

'garam masala'

In [35]:
fuzz.ratio('cheese', 'cheddar cheese')

60

In [38]:
classifying.aisle.unique()

array(['Produce', 'Canned and Jarred', 'Baking', 'Spices and Seasonings',
       'Ethnic Foods;Spices and Seasonings',
       'Canned and Jarred;Produce;Ethnic Foods',
       'Bakery/Bread;Ethnic Foods', 'Meat', 'Pasta and Rice', 'Beverages',
       'Pasta and Rice;Ethnic Foods;Health Foods',
       'Oil, Vinegar, Salad Dressing', 'Seafood', 'Ethnic Foods',
       'Condiments', 'Canned and Jarred;Milk, Eggs, Other Dairy',
       'Pasta and Rice;Canned and Jarred',
       'Produce;Spices and Seasonings', 'Milk, Eggs, Other Dairy',
       'Pasta and Rice;Health Foods', 'Refrigerated;Produce;Ethnic Foods',
       'Nuts;Savory Snacks', 'Produce;Ethnic Foods',
       'Produce;Ethnic Foods;Spices and Seasonings',
       'Health Foods;Baking', '?', 'Nuts',
       'Gluten Free;Health Foods;Baking', 'Bakery/Bread',
       'Gluten Free;Ethnic Foods;Health Foods', 'Cheese',
       'Ethnic Foods;Health Foods', None, 'Savory Snacks',
       'Nut butters, Jams, and Honey;Health Foods', 'Frozen',
   

In [68]:
whichaisle['broth']

['Canned and Jarred']